In [1]:
from adarank import AdaRank
from metrics import NDCGScorer,APScorer,DCGScorer

In [2]:
import pandas as pd
import numpy  as np

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import load_svmlight_file

In [3]:
df = pd.read_excel("data.xlsx", header=0)

In [4]:
tfidf = TfidfVectorizer().fit(df["features"])


HOLA002


In [5]:
df["features"] = df["features"].apply(lambda x : list(tfidf.transform([x]).indices))
df['id_doc'] =  df["id_doc"].apply(lambda x : [x])


HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002
HOLA002


In [6]:
df

,Qid,loinc_num,id_doc,rank,features
0,1,1003 3,[1],30,"[137, 124, 122, 121, 115, 108, 103, 102, 97, 7..."
1,1,10331 7,[2],17,"[140, 117, 72, 24, 23]"
2,1,1250 0,[3],33,"[121, 102, 81, 74, 70, 45]"
3,1,13317 3,[4],56,"[153, 144, 126, 125, 124, 116, 108, 98, 85, 76..."
4,1,14423 8,[5],58,"[151, 138, 134, 133, 83, 82, 72, 55, 54, 21]"
...,...,...,...,...,...
196,3,883 9,[63],44,"[140, 72, 63, 24, 23, 2]"
197,3,890 4,[64],6,"[122, 121, 119, 108, 103, 102, 97, 72, 63, 24,..."
198,3,925 8,[65],9,"[140, 110, 49, 27, 24]"
199,3,933 2,[66],27,"[140, 110, 27, 24]"


In [7]:
qid = df["Qid"].to_numpy()
X   = df["features"].to_list()
Y   = df["rank"].to_numpy()


In [8]:
X

[[137, 124, 122, 121, 115, 108, 103, 102, 97, 73, 72, 42, 15, 3],
 [140, 117, 72, 24, 23],
 [121, 102, 81, 74, 70, 45],
 [153, 144, 126, 125, 124, 116, 108, 98, 85, 76, 72, 46, 28, 17, 3],
 [151, 138, 134, 133, 83, 82, 72, 55, 54, 21],
 [142, 140, 110, 72, 63, 58, 27, 24, 23, 2],
 [151, 118, 105, 104, 88, 72, 61, 55, 54],
 [151, 122, 121, 118, 103, 102, 97, 88, 72, 61],
 [151, 122, 121, 118, 107, 106, 103, 102, 97, 88, 72, 68, 61, 1, 0],
 [151, 145, 118, 88, 72, 61],
 [151, 122, 121, 103, 102, 97, 72, 52, 33, 9, 6, 4],
 [151, 122, 121, 103, 102, 97, 83, 82, 72, 7],
 [122, 121, 113, 103, 102, 97, 84, 82, 72, 60, 7],
 [151, 122, 121, 103, 102, 97, 83, 82, 72, 29],
 [151, 122, 121, 103, 102, 97, 72, 52, 33, 12, 4],
 [132, 131, 76, 10],
 [132, 131, 129, 76, 10],
 [132, 131, 76, 34],
 [132, 131, 76, 39],
 [132, 131, 76, 59],
 [132, 131, 93, 76],
 [139, 132, 131, 130, 76],
 [146, 132, 131, 76],
 [151, 143, 122, 121, 103, 102, 97, 72, 38, 3],
 [151, 122, 121, 103, 102, 97, 72, 52, 33, 16, 9, 

In [9]:
max_len = max(df["features"].apply(lambda x: len(x)))

X_t = []
for i in range(len(X)):
    if X_t == []:
        X_t = np.array([X[i] + [0]*(max_len-len(X[i]))])
    else:
        X_t = np.append(X_t, [X[i] + [0]*(max_len-len(X[i]))], axis=0)
X_t       


C:\Users\Lucas\AppData\Local\Temp\ipykernel_16548\2020778632.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if X_t == []:


array([[137, 124, 122, ...,  15,   3,   0],
       [140, 117,  72, ...,   0,   0,   0],
       [121, 102,  81, ...,   0,   0,   0],
       ...,
       [140, 110,  49, ...,   0,   0,   0],
       [140, 110,  27, ...,   0,   0,   0],
       [142, 110,  95, ...,   0,   0,   0]])

In [10]:
from scipy import sparse


#Y = sparse.csr_matrix(Y)
#X = sparse.csr_matrix(X)

X = X_t

In [11]:
X = sparse.csr_matrix(X)

In [12]:
print(X)

  (0, 0)	137
  (0, 1)	124
  (0, 2)	122
  (0, 3)	121
  (0, 4)	115
  (0, 5)	108
  (0, 6)	103
  (0, 7)	102
  (0, 8)	97
  (0, 9)	73
  (0, 10)	72
  (0, 11)	42
  (0, 12)	15
  (0, 13)	3
  (1, 0)	140
  (1, 1)	117
  (1, 2)	72
  (1, 3)	24
  (1, 4)	23
  (2, 0)	121
  (2, 1)	102
  (2, 2)	81
  (2, 3)	74
  (2, 4)	70
  (2, 5)	45
  :	:
  (197, 2)	119
  (197, 3)	108
  (197, 4)	103
  (197, 5)	102
  (197, 6)	97
  (197, 7)	72
  (197, 8)	63
  (197, 9)	24
  (197, 10)	14
  (197, 11)	3
  (198, 0)	140
  (198, 1)	110
  (198, 2)	49
  (198, 3)	27
  (198, 4)	24
  (199, 0)	140
  (199, 1)	110
  (199, 2)	27
  (199, 3)	24
  (200, 0)	142
  (200, 1)	110
  (200, 2)	95
  (200, 3)	69
  (200, 4)	51
  (200, 5)	24


In [13]:
model = AdaRank(max_iter=1, scorer=DCGScorer(), verbose=True)
model.fit(X, Y , qid, X, Y , qid)

HOLA0
HOLA002
HOLA1
HOLA002
HOLA2
HOLA3
HOLA4
HOLA5 0
-1.0


ValueError: math domain error

In [ ]:
pred= model.predict(X, qid)

In [ ]:
min(pred)

inf